In [1]:
import pandas as pd 
import json 
import copy
import re 
from pathlib import Path
import os, sys

from collections import Counter, defaultdict
from copy import deepcopy
import pathlib

pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth',500)
from tqdm import tqdm
tqdm.pandas()
import string
from inflection import pluralize, singularize
from util_wordnet import get_sister_terms


In [2]:
def read_bert_vocab(bert_vocab_path = '../../data/bert-large-uncased-vocab.txt'):
    vocab = set()
    with open(bert_vocab_path, 'r') as fin: 
        lines = fin.readlines()
        for line in lines: 
            line = line.strip()
            vocab.add(line)
    return vocab 
bert_vocab = read_bert_vocab()

# Convert the data into LM probing format

In [3]:
def _get_article(word):
    if word[0] in ['a', 'e', 'i', 'o', 'u']:
        return 'an'
    return 'a'


def save_dict_to_json(examples, output_path):
    ''' 
    save a list of dicts into otuput_path, orient='records' (each line is a dict) 
    examples: a list of dicts
    output_path: 
    '''

    with open(output_path, 'w') as fout:
        for example in examples:
            json.dump(example, fout)
            fout.write("\n")
        print(f"save {output_path} with {len(examples)} lines")

def add_period_at_the_end_of_sentence(sentence):
    last_token = sentence[-1]
    if last_token != '.': 
        return sentence + '.'
    return [sentence]

# def process_lmdiag_data_to_lm():
#     data_dir = '../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/' 
#     files = ['singular.tsv', 'plural.tsv'] #, 'contextual.tsv']

#     for file in files: 
#         path = f"{data_dir}/{file}"
#         df = pd.read_csv(path, sep='\t', names=['masked_sentences', 'obj_label'])
#         df['masked_sentences'] =  df['masked_sentences'].apply(lambda x: add_period_at_the_end_of_sentence(x))
#         vocab = set(df['obj_label'].to_list())
#         label_counter = df['obj_label'].value_counts(normalize=True)
#         print(vocab)
#         if 'singular' in file: 
#              df['sub_label'] = df['masked_sentences'].apply(lambda x: x.split()[1])
#         elif 'plural' in file: 
#              df['sub_label'] = df['masked_sentences'].apply(lambda x: x.split()[0])
        
#         word_pairs = [name for name, group in df.groupby(['sub_label', 'obj_label'])]
#         hyper = set(df['obj_label'])
#         df['sub_label_pl'] =  df['sub_label'].apply(lambda x: pluralize(x))
#         df['obj_label'] =  df['obj_label'].apply(lambda x: [x])
#         df['masked_sentences'] =  df['masked_sentences'].apply(lambda x: [x])
#         df['relation'] = 'IsA'
#         df['sub_sister'] = df['sub_label'].apply(lambda x: get_sister_terms(x, distance_to_hypernym=6))
#         df['uuid'] = df.index + 1
#         display(df.head())
        
        
#         print(f"#Hypo {len(set(df['sub_label']))}")
#         print(f"#Hyper {len(hyper)}")
#         print(f"#Hypo-Hyper pairs:",len(word_pairs) )
#         print(label_counter)
#         display(label_counter.plot(kind='bar'))
        
        
#         out_file = file.replace(".tsv", "")
#         out_dir = f"{data_dir}/{out_file}/"
#         Path( out_dir ).mkdir( parents=True, exist_ok=True )
        
#         save_dict_to_json(examples=df.to_dict(orient='records'), output_path=out_dir + 'IsA.jsonl')     
# process_lmdiag_data_to_lm()

In [4]:
# !scp Syntagmatic/LM-Diagnostic-Extended/plural/IsA.jsonl spartan:/home/chunhua/cogsci/DAP/data/lm_diagnostic_extended/plural/IsA.jsonl
# !scp ../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/singular/IsA.jsonl spartan:/home/chunhua/cogsci/DAP/data/lm_diagnostic_extended/singular/IsA.jsonl
# !scp ../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/sgpl/IsA.jsonl spartan:/home/chunhua/cogsci/DAP/data/lm_diagnostic_extended/sgpl/IsA.jsonl
    

In [5]:
# !scp spartan:~/cogsci/DAP/log/anchors/lm_diagnostic_extended.anchors.csv ../log/bert-large-uncased/lm_diagnostic_extended/

# Prcess data for consitency pair check

1. DEF-SAP '
`A(a) X is a(n) Y.`


2. X are Y.
DEF-DAP

A(n) X or Z is a(n) Y.
X or Z are Y.

LSP-SAP
Y such as X.
Y such as X.
LSP-DAP
Y such as X or Z.
Y such as X or Z.


In [6]:
def get_masked_sentence_definitional(x, y, singluar_x=True):
    if singluar_x: 
        return f"{_get_article(x)} {x} is {_get_article(y)} [MASK]."
    else: 
         return f"{x} are [MASK]."
  
      
def load_json_to_df(filepath):
    with open(filepath, 'r', encoding='utf-8') as fin:
        data = fin.readlines()
        data = [eval(x) for x in data]
        df = pd.DataFrame(data)
    return df 

def definition_sap_singular_plural(filepath):
    '''
    the dataformat is the *.jsonl file we used to probe LMs
    '''
    if '.jsonl' in filepath:
        df = load_json_to_df(filepath)
        df['sub_label_singular'] = df['sub_label'].apply(lambda x: singularize(x))
        df['sub_label_plural'] = df['sub_label'].apply(lambda x: pluralize(x))
    elif ".csv" in filepath:
        df = pd.read_csv(filepath)
        for col in ['obj_label']:
            df[col] = df[col].apply(lambda x: eval(x))
        df['sub_label_singular'] = df['sub_label_sg'] #.apply(lambda x: singularize(x))
        df['sub_label_plural'] = df['sub_label_pl'] #.apply(lambda x: pluralize(x))
    
    df['obj_label'] = df['obj_label'].apply(lambda x: x[0])

    df['mask_sentences_singular'] =  df[['sub_label_singular', 'obj_label']].apply(lambda x: get_masked_sentence_definitional(x[0], x[1], singluar_x = True), axis=1 )
    df['mask_sentences_plural'] =  df[['sub_label_plural', 'obj_label']].apply(lambda x: get_masked_sentence_definitional(x[0], x[1], singluar_x = False), axis=1)

    df['relation'] = 'IsA'
    df['uuid'] = df.index + 1
    df['obj_label_singular'] =  df['obj_label'].apply(lambda x: [singularize(x)])
    df['obj_label_plural'] =  df['obj_label'].apply(lambda x: [pluralize(x)])


    df = df[['sub_label_singular', 'obj_label_singular', 'mask_sentences_singular', 
             'sub_label_plural', 'obj_label_plural', 'mask_sentences_plural', 
             'uuid', 'relation', 'obj_label']]
    #     display(df.head())
    #     out_file = file.replace(".tsv", "")
    #     out_dir = f"{data_dir}/{out_file}/"
    #     Path( out_dir ).mkdir( parents=True, exist_ok=True )
    #     save_dict_to_json(examples=df.to_dict(orient='records'), output_path=out_dir+"IsA.jsonl") 
    return df 


# df = definition_sap_singular_plural('../data/BLESS/hypernym/IsA.jsonl')

# data_path = dataset_to_localpath['BLESS']
# print(data_path)
# df = definition_sap_singular_plural(data_path)
# df.head()
# df.query("sub_label_singular=='saw'")

In [7]:

def definition_dap_singular_plural(df):
    df['mask_sentences_singular'] = df[['sub_label_singular', 'mask_sentences_singular']].apply(lambda x: x[1].replace(x[0], f"{x[0]} or [Z]"), axis=1)
    df['mask_sentences_plural'] = df[['sub_label_plural', 'mask_sentences_plural']].apply(lambda x: x[1].replace(x[0], f"{x[0]} or [Z]"), axis=1)

    df = df[['sub_label_singular', 'obj_label_singular', 'mask_sentences_singular', 
            'sub_label_plural', 'obj_label_plural', 'mask_sentences_plural', 
             'uuid', 'relation']]

    return df 

def lsp_sap_singular_plural(df):
    '''
    sap: Y such as X 

    '''
    
    df['mask_sentences_singular'] = df['sub_label_singular'].apply(lambda x: f"[MASK] such as {_get_article(x)} {x}.")
   
    df['mask_sentences_plural'] = df['sub_label_plural'].apply(lambda x:f"[MASK] such as {x}.")

    df = df[['sub_label_singular', 'obj_label_singular', 'mask_sentences_singular', 
            'sub_label_plural', 'obj_label_plural', 'mask_sentences_plural', 
             'uuid', 'relation']]

    return df 


def lsp_dap_singular_plural(df):
    '''
    sap: Y such as X 

    '''
    df['mask_sentences_singular'] = df['sub_label_singular'].apply(lambda x: f"[MASK] such as {_get_article(x)} {x} or [Z].")

    df['mask_sentences_plural'] = df['sub_label_plural'].apply(lambda x:f"[MASK] such as {x} or [Z].")

    df = df[['sub_label_singular', 'obj_label_singular', 'mask_sentences_singular', 
            'sub_label_plural', 'obj_label_plural', 'mask_sentences_plural', 
             'uuid', 'relation']]

    return df 


In [8]:
from inflection import singularize, pluralize

def read_anchors(path_sg, path_pl, anchor_source, debug=False):
    '''
    read the anchor files mined from singualr and plural
    
    args: 
        anchor_soure: using the anchors mined from singular probe or plural probe
        
    return: 
        dic_sub_to_anchors_singular: both sub_label and subj_anchors are singular 
        dic_sub_to_anchors_plural: both sub_label and subj_anchors are plural 
    '''
#     dfsg = pd.read_csv(path_sg)
#     dfsg['subj_anchors'] = dfsg['subj_anchors'].apply(lambda x: eval(x))
    
#     dfpl = pd.read_csv(path_pl)
#     dfpl['subj_anchors'] = dfpl['subj_anchors'].apply(lambda x: eval(x))
#     df = pd.merge(dfsg, dfpl, on = 'uuid', suffixes=('_sg', '_pl'))
# if anchor_source == 'plural':
#         #convert the singular anchors into singular format
#         df['subj_anchors_sg'] = df['subj_anchors_pl'].progress_apply(lambda x: [singularize(word) for word in x])
#     elif anchor_source == 'singular':
#         #convert the plural anchors into singular format
#         df['subj_anchors_pl'] = df['subj_anchors_sg'].progress_apply(lambda x: [pluralize(word) for word in x])

    df = pd.read_csv(path_pl)
    
    if debug: df = df.head(5)
    df['subj_anchors_sg'] = df['subj_anchors_sg'].apply(lambda x: eval(x))
    df['subj_anchors_pl'] = df['subj_anchors_pl'].apply(lambda x: eval(x))
        
    dic_sub_to_anchors_singular = dict(zip(df['sub_label_sg'], df['subj_anchors_sg']))
    dic_sub_to_anchors_plural = dict(zip(df['sub_label_pl'], df['subj_anchors_pl']))
    
    return dic_sub_to_anchors_singular, dic_sub_to_anchors_plural

def insert_anchors(dic_sub_to_anchors, df, mask_col, sub_col, anchor_col, probe_type, article_for_z=False):
    
    df[anchor_col] = df[sub_col].apply(lambda x: dic_sub_to_anchors.get(x) )
#     display(df[mask_col].head())
    
    if probe_type =='plural':
        df[mask_col] =  df[[anchor_col, mask_col]].apply(lambda x: [ x[1].replace('[Z]', anchor)  for anchor in x[0]], axis=1)
    elif probe_type == 'singular':
        if article_for_z: 
           df[mask_col] =  df[[anchor_col, mask_col]].apply(lambda x: [ x[1].replace('[Z]', "{} {}".format(_get_article(anchor), anchor))  for anchor in x[0]], axis=1) 
        else:
#             display(df.head())
#             for anchor_col, mask_col in zip(df[anchor_col], df[mask_col]):
#                 print(anchor_col, mask_col)
#                 print([mask_col.replace('[Z]', anchor)  for anchor in anchor_col])
            
            df[mask_col] =  df[[anchor_col, mask_col]].apply(lambda x: [ x[1].replace('[Z]', anchor)  for anchor in x[0]], axis=1) 
            
    return df 


def save_hypernym_vocab(df, vocab_path, y_singular=True, y_plural=True):
    vocab_sg = set(x[0] for x in df['obj_label_singular'])
    vocab_pl = set(x[0] for x in df_def_sap['obj_label_plural'])
    
    if y_singular and y_plural: 
        vocab = vocab_sg.union(vocab_pl)
        df_vocab = pd.DataFrame(data=list(vocab))
        with open(vocab_path, 'w') as fout:
            df_vocab.to_csv(vocab_path, header=None, index=None, sep=' ', mode='a')
    print(f"save {vocab_path}")
    


In [9]:
from utils_path import dataset_to_respath

def get_dataset_to_respath(dataset_to_respath, print_flag=False):
    # remote path 
    #dataset_to_respath = {'hypernymsuite-BLESS': 'log/bert-large-uncased/hypernymsuite/BLESS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'lm_diagnostic_extended-singular': 'log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv', 'clsb-singular': 'log/bert-large-uncased/clsb/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.CLSB.csv', 'hypernymsuite-LEDS': 'log/bert-large-uncased/hypernymsuite/LEDS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'hypernymsuite-EVAL': 'log/bert-large-uncased/hypernymsuite/EVAL/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'hypernymsuite-SHWARTZ': 'log/bert-large-uncased/hypernymsuite/SHWARTZ/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv'}
    
#     dataset_to_respath={
#     "hypernymsuite-BLESS": "log/bert-large-uncased/hypernymsuite/BLESS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.HYPERNYMSUITE.csv",
#     "lm_diagnostic_extended-singular": "log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv",
#     "clsb-singular": "log/bert-large-uncased/clsb/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.CLSB.csv",
#     "hypernymsuite-LEDS": "log/bert-large-uncased/hypernymsuite/LEDS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.HYPERNYMSUITE.csv",
#     "hypernymsuite-EVAL": "log/bert-large-uncased/hypernymsuite/EVAL/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.HYPERNYMSUITE.csv",
#     "hypernymsuite-SHWARTZ": "log/bert-large-uncased/hypernymsuite/SHWARTZ/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.HYPERNYMSUITE.csv"
#     }
    source_dir = 'spartan:~/cogsci/DAP/'
    target_dir = '../../'
    dataset_to_localpath = defaultdict()
    dataset_rename = {
        'hypernymsuite-BLESS': 'BLESS', 'lm_diagnostic_extended-singular': 'DIAG', 'clsb-singular':'CLSB', 'hypernymsuite-LEDS': 'LEDS', 'hypernymsuite-EVAL': 'EVAL', 'hypernymsuite-SHWARTZ': 
        "SHWARTZ"
    }
    dataset_name_to_relpath = defaultdict()
    for dataset, path in dataset_to_respath.items():
        path = path.replace(".tsv", ".csv")
        source_path = source_dir + path 
        dataset_l1 = dataset.split("-")[0]
        dataset_l2 = dataset.split("-")[1] 
        target_path = target_dir + path
        
        scp_string = f"!scp {source_path} {target_path}"
        if print_flag:
            print(scp_string)
            print()
#         print(target_path)
        dataset_to_localpath[dataset_rename[dataset]] = target_path 
        dataset_name_to_relpath[dataset_rename[dataset]] = "/".join(dataset.split('-'))
#     print(dataset_to_localpath)
    return dataset_to_localpath, dataset_name_to_relpath
dataset_to_localpath, dataset_name_to_relpath = get_dataset_to_respath(dataset_to_respath)
print(json.dumps(dataset_to_localpath, indent=4))
# print(dataset_name_to_relpath)

{
    "BLESS": "../../log/bert-large-uncased/hypernymsuite/BLESS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.HYPERNYMSUITE.csv",
    "DIAG": "../../log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv",
    "CLSB": "../../log/bert-large-uncased/clsb/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.CLSB.csv",
    "LEDS": "../../log/bert-large-uncased/hypernymsuite/LEDS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_fil

In [10]:
# path = "../../log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv"
# df = pd.read_csv(path)
# df.query("sub_label_sg == 'swordtail'")[['subj_anchors_sg']]

In [11]:

# dataset_to_localpath, dataset_name_to_relpath = get_dataset_to_respath(dataset_to_respath)

dataset_to_localpath = {"LM": "../../log/bert-large-uncased/ALL/swow_rw/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_LM_swow_score_source_None.HYPERNYMSUITE.csv",
"ShareSWOWStrength": "../../log/bert-large-uncased/ALL/swow_rw/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_LM_swow_score_source_ShareSWOWStrength.HYPERNYMSUITE.csv",
"ShareSWOWSimilar": "../../log/bert-large-uncased/ALL/swow_rw/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_LM_swow_score_source_ShareSWOWSimilar.HYPERNYMSUITE.csv",
"AddSWOWSimilar": "../../log/bert-large-uncased/ALL/swow_rw/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_LM_swow_score_source_AddSWOWSimilar.HYPERNYMSUITE.csv",
"AddSWOWStrength": "../../log/bert-large-uncased/ALL/swow_rw/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_LM_swow_score_source_AddSWOWStrength.HYPERNYMSUITE.csv"
}

for dataset, data_path in dataset_to_localpath.items():
    dic_sub_to_anchors_singular,dic_sub_to_anchors_plural = read_anchors(data_path, data_path, anchor_source='plural', debug=False)
 
    #out_dir = f'../../data/{dataset_name_to_relpath[dataset]}/consistency/'
    out_dir = f'../../data/hypernymsuite/ALL/{dataset}/consistency_rw/'
    
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)
    vocab_path = out_dir + 'vocab.txt'
    
    df_def_sap = definition_sap_singular_plural(data_path)
    df_def_sap['obj_in_BERT'] = df_def_sap['obj_label'].apply(lambda x: 1 if x in bert_vocab else 0)
    df_def_sap = df_def_sap.query("obj_in_BERT ==1") 
    
    df_def_dap = definition_dap_singular_plural(deepcopy(df_def_sap))
    df_lsp_sap = lsp_sap_singular_plural(deepcopy(df_def_sap))
    df_lsp_dap = lsp_dap_singular_plural(deepcopy(df_def_sap))

    df_def_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_singular, df= df_def_dap, mask_col = 'mask_sentences_singular', sub_col = 'sub_label_singular', anchor_col='subj_anchors_singular', probe_type='singular', article_for_z=False)

    df_lsp_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_singular, df= df_lsp_dap, mask_col = 'mask_sentences_singular', sub_col = 'sub_label_singular', anchor_col='subj_anchors_singular', probe_type = 'singular', article_for_z=False)

    df_lsp_dap[['sub_label_singular', 'subj_anchors_singular', 'mask_sentences_singular']].head() #''



    # dic_sub_to_anchors_plural = read_anchors(path_anchor_pl)
    df_def_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_plural, df= df_def_dap, mask_col = 'mask_sentences_plural', sub_col = 'sub_label_plural', anchor_col='subj_anchors_plural', probe_type='plural')

    df_lsp_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_plural, df= df_lsp_dap, mask_col = 'mask_sentences_plural', sub_col = 'sub_label_plural', anchor_col='subj_anchors_plural', probe_type='plural')

    df_lsp_dap[['sub_label_plural', 'subj_anchors_plural', 'mask_sentences_plural']].head() #''


    # save files 
    # out_dir = '../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/consistency/'

    save_dict_to_json(examples=df_def_sap.to_dict(orient='records'), output_path=out_dir + 'IsA.def_sap.jsonl') 
    save_dict_to_json(examples=df_def_dap.to_dict(orient='records'), output_path=out_dir + 'IsA.def_dap.jsonl') 
    save_dict_to_json(examples=df_lsp_sap.to_dict(orient='records'), output_path=out_dir + 'IsA.lsp_sap.jsonl') 
    save_dict_to_json(examples=df_lsp_dap.to_dict(orient='records'), output_path=out_dir + 'IsA.lsp_dap.jsonl') 
    save_hypernym_vocab(df_def_sap, vocab_path,  y_singular=True, y_plural=True)

    df_def_sap.to_csv(out_dir + 'IsA.def_sap.csv') 
    df_def_dap.to_csv(out_dir + 'IsA.def_dap.csv') 
    df_lsp_sap.to_csv(out_dir + 'IsA.lsp_sap.csv') 
    df_lsp_dap.to_csv(out_dir + 'IsA.lsp_dap.csv')
    display(df_def_sap.head())
    display(df_def_dap.head())
    display(df_lsp_sap.head())
    display(df_lsp_dap.head())



save ../../data/hypernymsuite/ALL/LM/consistency_rw/IsA.def_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/LM/consistency_rw/IsA.def_dap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/LM/consistency_rw/IsA.lsp_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/LM/consistency_rw/IsA.lsp_dap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/LM/consistency_rw/vocab.txt


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,obj_label,obj_in_BERT
0,catfish,[fish],a catfish is a [MASK].,catfish,[fishes],catfish are [MASK].,1,IsA,fish,1
1,lemon,[produce],a lemon is a [MASK].,lemon,[produce],lemon are [MASK].,2,IsA,produce,1
2,turtle,[beast],a turtle is a [MASK].,turtles,[beasts],turtles are [MASK].,3,IsA,beast,1
3,bag,[luggage],a bag is a [MASK].,bags,[luggages],bags are [MASK].,4,IsA,luggage,1
4,rabbit,[creature],a rabbit is a [MASK].,rabbits,[creatures],rabbits are [MASK].,5,IsA,creature,1


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,subj_anchors_singular,subj_anchors_plural
0,catfish,[fish],"[a catfish or bass is a [MASK]., a catfish or carp is a [MASK]., a catfish or shrimp is a [MASK]., a catfish or perch is a [MASK]., a catfish or alligator is a [MASK].]",catfish,[fishes],"[catfish or basses are [MASK]., catfish or carp are [MASK]., catfish or shrimps are [MASK]., catfish or perches are [MASK]., catfish or alligators are [MASK].]",1,IsA,"[bass, carp, shrimp, perch, alligator]","[basses, carp, shrimps, perches, alligators]"
1,lemon,[produce],"[a lemon or lime is a [MASK]., a lemon or orange is a [MASK]., a lemon or apple is a [MASK]., a lemon or yes is a [MASK]., a lemon or cherry is a [MASK].]",lemon,[produce],"[lemon or limes are [MASK]., lemon or orange are [MASK]., lemon or apples are [MASK]., lemon or yes are [MASK]., lemon or cherries are [MASK].]",2,IsA,"[lime, orange, apple, yes, cherry]","[limes, orange, apples, yes, cherries]"
2,turtle,[beast],"[a turtle or snake is a [MASK]., a turtle or bird is a [MASK]., a turtle or fish is a [MASK]., a turtle or lizard is a [MASK]., a turtle or frog is a [MASK].]",turtles,[beasts],"[turtles or snakes are [MASK]., turtles or birds are [MASK]., turtles or fishes are [MASK]., turtles or lizards are [MASK]., turtles or frogs are [MASK].]",3,IsA,"[snake, bird, fish, lizard, frog]","[snakes, birds, fishes, lizards, frogs]"
3,bag,[luggage],"[a bag or box is a [MASK]., a bag or yes is a [MASK]., a bag or shoe is a [MASK]., a bag or case is a [MASK]., a bag or obviously is a [MASK].]",bags,[luggages],"[bags or boxes are [MASK]., bags or yes are [MASK]., bags or shoes are [MASK]., bags or cases are [MASK]., bags or obviously are [MASK].]",4,IsA,"[box, yes, shoe, case, obviously]","[boxes, yes, shoes, cases, obviously]"
4,rabbit,[creature],"[a rabbit or fox is a [MASK]., a rabbit or deer is a [MASK]., a rabbit or squirrels is a [MASK]., a rabbit or mouse is a [MASK]., a rabbit or rodent is a [MASK].]",rabbits,[creatures],"[rabbits or foxes are [MASK]., rabbits or deer are [MASK]., rabbits or squirrels are [MASK]., rabbits or mice are [MASK]., rabbits or rodents are [MASK].]",5,IsA,"[fox, deer, squirrels, mouse, rodent]","[foxes, deer, squirrels, mice, rodents]"


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation
0,catfish,[fish],[MASK] such as a catfish.,catfish,[fishes],[MASK] such as catfish.,1,IsA
1,lemon,[produce],[MASK] such as a lemon.,lemon,[produce],[MASK] such as lemon.,2,IsA
2,turtle,[beast],[MASK] such as a turtle.,turtles,[beasts],[MASK] such as turtles.,3,IsA
3,bag,[luggage],[MASK] such as a bag.,bags,[luggages],[MASK] such as bags.,4,IsA
4,rabbit,[creature],[MASK] such as a rabbit.,rabbits,[creatures],[MASK] such as rabbits.,5,IsA


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,subj_anchors_singular,subj_anchors_plural
0,catfish,[fish],"[[MASK] such as a catfish or bass., [MASK] such as a catfish or carp., [MASK] such as a catfish or shrimp., [MASK] such as a catfish or perch., [MASK] such as a catfish or alligator.]",catfish,[fishes],"[[MASK] such as catfish or basses., [MASK] such as catfish or carp., [MASK] such as catfish or shrimps., [MASK] such as catfish or perches., [MASK] such as catfish or alligators.]",1,IsA,"[bass, carp, shrimp, perch, alligator]","[basses, carp, shrimps, perches, alligators]"
1,lemon,[produce],"[[MASK] such as a lemon or lime., [MASK] such as a lemon or orange., [MASK] such as a lemon or apple., [MASK] such as a lemon or yes., [MASK] such as a lemon or cherry.]",lemon,[produce],"[[MASK] such as lemon or limes., [MASK] such as lemon or orange., [MASK] such as lemon or apples., [MASK] such as lemon or yes., [MASK] such as lemon or cherries.]",2,IsA,"[lime, orange, apple, yes, cherry]","[limes, orange, apples, yes, cherries]"
2,turtle,[beast],"[[MASK] such as a turtle or snake., [MASK] such as a turtle or bird., [MASK] such as a turtle or fish., [MASK] such as a turtle or lizard., [MASK] such as a turtle or frog.]",turtles,[beasts],"[[MASK] such as turtles or snakes., [MASK] such as turtles or birds., [MASK] such as turtles or fishes., [MASK] such as turtles or lizards., [MASK] such as turtles or frogs.]",3,IsA,"[snake, bird, fish, lizard, frog]","[snakes, birds, fishes, lizards, frogs]"
3,bag,[luggage],"[[MASK] such as a bag or box., [MASK] such as a bag or yes., [MASK] such as a bag or shoe., [MASK] such as a bag or case., [MASK] such as a bag or obviously.]",bags,[luggages],"[[MASK] such as bags or boxes., [MASK] such as bags or yes., [MASK] such as bags or shoes., [MASK] such as bags or cases., [MASK] such as bags or obviously.]",4,IsA,"[box, yes, shoe, case, obviously]","[boxes, yes, shoes, cases, obviously]"
4,rabbit,[creature],"[[MASK] such as a rabbit or fox., [MASK] such as a rabbit or deer., [MASK] such as a rabbit or squirrels., [MASK] such as a rabbit or mouse., [MASK] such as a rabbit or rodent.]",rabbits,[creatures],"[[MASK] such as rabbits or foxes., [MASK] such as rabbits or deer., [MASK] such as rabbits or squirrels., [MASK] such as rabbits or mice., [MASK] such as rabbits or rodents.]",5,IsA,"[fox, deer, squirrels, mouse, rodent]","[foxes, deer, squirrels, mice, rodents]"


...

save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw/IsA.def_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw/IsA.def_dap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw/IsA.lsp_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw/IsA.lsp_dap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw/vocab.txt


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,obj_label,obj_in_BERT
0,catfish,[fish],a catfish is a [MASK].,catfish,[fishes],catfish are [MASK].,1,IsA,fish,1
1,lemon,[produce],a lemon is a [MASK].,lemon,[produce],lemon are [MASK].,2,IsA,produce,1
2,turtle,[beast],a turtle is a [MASK].,turtles,[beasts],turtles are [MASK].,3,IsA,beast,1
3,bag,[luggage],a bag is a [MASK].,bags,[luggages],bags are [MASK].,4,IsA,luggage,1
4,rabbit,[creature],a rabbit is a [MASK].,rabbits,[creatures],rabbits are [MASK].,5,IsA,creature,1


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,subj_anchors_singular,subj_anchors_plural
0,catfish,[fish],"[a catfish or bass is a [MASK]., a catfish or carp is a [MASK]., a catfish or shrimp is a [MASK]., a catfish or perch is a [MASK]., a catfish or alligator is a [MASK].]",catfish,[fishes],"[catfish or basses are [MASK]., catfish or carp are [MASK]., catfish or shrimps are [MASK]., catfish or perches are [MASK]., catfish or alligators are [MASK].]",1,IsA,"[bass, carp, shrimp, perch, alligator]","[basses, carp, shrimps, perches, alligators]"
1,lemon,[produce],"[a lemon or lime is a [MASK]., a lemon or orange is a [MASK]., a lemon or yellow is a [MASK]., a lemon or apple is a [MASK]., a lemon or yes is a [MASK].]",lemon,[produce],"[lemon or limes are [MASK]., lemon or orange are [MASK]., lemon or yellow are [MASK]., lemon or apples are [MASK]., lemon or yes are [MASK].]",2,IsA,"[lime, orange, yellow, apple, yes]","[limes, orange, yellow, apples, yes]"
2,turtle,[beast],"[a turtle or snake is a [MASK]., a turtle or bird is a [MASK]., a turtle or fish is a [MASK]., a turtle or lizard is a [MASK]., a turtle or frog is a [MASK].]",turtles,[beasts],"[turtles or snakes are [MASK]., turtles or birds are [MASK]., turtles or fishes are [MASK]., turtles or lizards are [MASK]., turtles or frogs are [MASK].]",3,IsA,"[snake, bird, fish, lizard, frog]","[snakes, birds, fishes, lizards, frogs]"
3,bag,[luggage],"[a bag or box is a [MASK]., a bag or paper is a [MASK]., a bag or yes is a [MASK]., a bag or shoe is a [MASK]., a bag or case is a [MASK].]",bags,[luggages],"[bags or boxes are [MASK]., bags or papers are [MASK]., bags or yes are [MASK]., bags or shoes are [MASK]., bags or cases are [MASK].]",4,IsA,"[box, paper, yes, shoe, case]","[boxes, papers, yes, shoes, cases]"
4,rabbit,[creature],"[a rabbit or fox is a [MASK]., a rabbit or deer is a [MASK]., a rabbit or squirrels is a [MASK]., a rabbit or mouse is a [MASK]., a rabbit or rodent is a [MASK].]",rabbits,[creatures],"[rabbits or foxes are [MASK]., rabbits or deer are [MASK]., rabbits or squirrels are [MASK]., rabbits or mice are [MASK]., rabbits or rodents are [MASK].]",5,IsA,"[fox, deer, squirrels, mouse, rodent]","[foxes, deer, squirrels, mice, rodents]"


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation
0,catfish,[fish],[MASK] such as a catfish.,catfish,[fishes],[MASK] such as catfish.,1,IsA
1,lemon,[produce],[MASK] such as a lemon.,lemon,[produce],[MASK] such as lemon.,2,IsA
2,turtle,[beast],[MASK] such as a turtle.,turtles,[beasts],[MASK] such as turtles.,3,IsA
3,bag,[luggage],[MASK] such as a bag.,bags,[luggages],[MASK] such as bags.,4,IsA
4,rabbit,[creature],[MASK] such as a rabbit.,rabbits,[creatures],[MASK] such as rabbits.,5,IsA


,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,subj_anchors_singular,subj_anchors_plural
0,catfish,[fish],"[[MASK] such as a catfish or bass., [MASK] such as a catfish or carp., [MASK] such as a catfish or shrimp., [MASK] such as a catfish or perch., [MASK] such as a catfish or alligator.]",catfish,[fishes],"[[MASK] such as catfish or basses., [MASK] such as catfish or carp., [MASK] such as catfish or shrimps., [MASK] such as catfish or perches., [MASK] such as catfish or alligators.]",1,IsA,"[bass, carp, shrimp, perch, alligator]","[basses, carp, shrimps, perches, alligators]"
1,lemon,[produce],"[[MASK] such as a lemon or lime., [MASK] such as a lemon or orange., [MASK] such as a lemon or yellow., [MASK] such as a lemon or apple., [MASK] such as a lemon or yes.]",lemon,[produce],"[[MASK] such as lemon or limes., [MASK] such as lemon or orange., [MASK] such as lemon or yellow., [MASK] such as lemon or apples., [MASK] such as lemon or yes.]",2,IsA,"[lime, orange, yellow, apple, yes]","[limes, orange, yellow, apples, yes]"
2,turtle,[beast],"[[MASK] such as a turtle or snake., [MASK] such as a turtle or bird., [MASK] such as a turtle or fish., [MASK] such as a turtle or lizard., [MASK] such as a turtle or frog.]",turtles,[beasts],"[[MASK] such as turtles or snakes., [MASK] such as turtles or birds., [MASK] such as turtles or fishes., [MASK] such as turtles or lizards., [MASK] such as turtles or frogs.]",3,IsA,"[snake, bird, fish, lizard, frog]","[snakes, birds, fishes, lizards, frogs]"
3,bag,[luggage],"[[MASK] such as a bag or box., [MASK] such as a bag or paper., [MASK] such as a bag or yes., [MASK] such as a bag or shoe., [MASK] such as a bag or case.]",bags,[luggages],"[[MASK] such as bags or boxes., [MASK] such as bags or papers., [MASK] such as bags or yes., [MASK] such as bags or shoes., [MASK] such as bags or cases.]",4,IsA,"[box, paper, yes, shoe, case]","[boxes, papers, yes, shoes, cases]"
4,rabbit,[creature],"[[MASK] such as a rabbit or fox., [MASK] such as a rabbit or deer., [MASK] such as a rabbit or squirrels., [MASK] such as a rabbit or mouse., [MASK] such as a rabbit or rodent.]",rabbits,[creatures],"[[MASK] such as rabbits or foxes., [MASK] such as rabbits or deer., [MASK] such as rabbits or squirrels., [MASK] such as rabbits or mice., [MASK] such as rabbits or rodents.]",5,IsA,"[fox, deer, squirrels, mouse, rodent]","[foxes, deer, squirrels, mice, rodents]"


In [10]:
display (df_def_sap)

,sub_label_singular,obj_label_singular,mask_sentences_singular,sub_label_plural,obj_label_plural,mask_sentences_plural,uuid,relation,obj_label,obj_in_BERT
0,catfish,[fish],a catfish is a [MASK].,catfish,[fishes],catfish are [MASK].,1,IsA,fish,1
1,lemon,[produce],a lemon is a [MASK].,lemon,[produce],lemon are [MASK].,2,IsA,produce,1
2,turtle,[beast],a turtle is a [MASK].,turtles,[beasts],turtles are [MASK].,3,IsA,beast,1
3,bag,[luggage],a bag is a [MASK].,bags,[luggages],bags are [MASK].,4,IsA,luggage,1
4,rabbit,[creature],a rabbit is a [MASK].,rabbits,[creatures],rabbits are [MASK].,5,IsA,creature,1
...,...,...,...,...,...,...,...,...,...,...
4768,cherry,[band],a cherry is a [MASK].,cherries,[bands],cherries are [MASK].,4769,IsA,band,1
4769,boat,[organization],a boat is an [MASK].,boats,[organizations],boats are [MASK].,4770,IsA,organization,1
4770,phoenix,[constellation],a phoenix is a [MASK].,phoenix,[constellations],phoenix are [MASK].,4771,IsA,constellation,1
4771,path,[line],a path is a [MASK].,paths,[lines],paths are [MASK].,4772,IsA,line,1


In [13]:
# !scp -r ../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/consistency spartan:/home/chunhua/cogsci/DAP/data/lm_diagnostic_extended/
# # !ls ../
# # df_lsp_sap.head()
# # df_lsp_dap.head()

# Prepare data for checking group consistency 

## DFP (group consistency)

In [12]:
def_sap = [
         "[X] is a [Y].", 
         "[X] is a type of [Y].", 
         "[X] is a kind of [Y].", 
        ]

def_dap = [
         "[X] or [Z] is a [Y].", 
         "[X] or [Z] is a type of [Y].", 
         "[X] or [Z] is a kind of [Y]."]

def insert_anchors(dic_sub_to_anchors, df, mask_col, sub_col, anchor_col):
    df[mask_col].head()
    df[anchor_col] = df[sub_col].apply(lambda x: dic_sub_to_anchors.get(x) )
    df[mask_col] =  df[[anchor_col, mask_col]].apply(lambda x: [ x[1].replace('[Z]', anchor)  for anchor in x[0]], axis=1)
    return df

def fill_x_into_patterns(df, patterns, ):
    '''
    sap: Y such as X 

    '''
    
    for (i, pattern) in enumerate(patterns):
        pattern_id = i+1
        df[f'mask_sentences_singular_{pattern_id}'] = df['sub_label_singular'].apply(lambda x: pattern.replace("[X]", f"{_get_article(x)} {x}").replace("[Y]", '[MASK]'))
        df[f'mask_sentences_plural_{pattern_id}'] = df['sub_label_plural'].apply(lambda x: pattern.replace("[X]", x).replace("[Y]", '[MASK]'))
        
    mask_sentences_sg_cols = [x for x in df.columns if 'mask_sentences_singular_' in x]
    mask_sentences_pl_cols = [x for x in df.columns if 'mask_sentences_plural_' in x]
    return df, mask_sentences_sg_cols, mask_sentences_pl_cols


for dataset, data_path in dataset_to_localpath.items():
    dic_sub_to_anchors_singular,dic_sub_to_anchors_plural = read_anchors(data_path, data_path, anchor_source='plural', debug=False)

    
#     out_dir = f'../../data/{dataset_name_to_relpath[dataset]}/consistency_group/'
    out_dir = f'../../data/hypernymsuite/ALL/{dataset}/consistency_rw_group/'
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)
    vocab_path = out_dir + 'vocab.txt'
    
    df_def_sap = definition_sap_singular_plural(data_path)
    
    df_def_sap['obj_in_BERT'] = df_def_sap['obj_label'].apply(lambda x: 1 if x in bert_vocab else 0)
    df_def_sap = df_def_sap.query("obj_in_BERT ==1") 
    
    df_def_sap,mask_sentences_sg_cols, mask_sentences_pl_cols = fill_x_into_patterns(deepcopy(df_def_sap), patterns=def_sap)
    df_def_dap,mask_sentences_sg_cols, mask_sentences_pl_cols = fill_x_into_patterns(deepcopy(df_def_sap), patterns=def_dap)

    # fill Z with real anchors 
    for mask_col in mask_sentences_sg_cols:
        df_def_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_singular, df= df_def_dap, mask_col = mask_col, sub_col = 'sub_label_singular', anchor_col='subj_anchors_singular')

    for mask_col in mask_sentences_pl_cols:
        df_def_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_plural, df= df_def_dap, mask_col = mask_col, sub_col = 'sub_label_plural', anchor_col='subj_anchors_plural')

    # # save files 
    output_cols = ['sub_label_singular', 'obj_label_singular','sub_label_plural', 'obj_label_plural',  'uuid',
           'relation', 
           'mask_sentences_singular_1', 'mask_sentences_singular_2', 'mask_sentences_singular_3', 
           'mask_sentences_plural_1', 'mask_sentences_plural_2', 'mask_sentences_plural_3']

    df_def_sap = df_def_sap[output_cols]
    df_def_dap = df_def_dap[output_cols]
    
#     out_dir = '../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/consistency_group/'
    save_dict_to_json(examples=df_def_sap.to_dict(orient='records'), output_path=out_dir + 'IsA.def_sap.jsonl') 
    save_dict_to_json(examples=df_def_dap.to_dict(orient='records'), output_path=out_dir + 'IsA.def_dap.jsonl') 
    df_def_sap.to_csv(out_dir + 'IsA.def_sap.csv')
    df_def_dap.to_csv(out_dir + 'IsA.def_dap.csv')
    display(df_def_sap.head())
    display(df_def_dap.head())

save ../../data/hypernymsuite/ALL/LM/consistency_rw_group/IsA.def_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/LM/consistency_rw_group/IsA.def_dap.jsonl with 2094 lines


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,a catfish is a [MASK].,a catfish is a type of [MASK].,a catfish is a kind of [MASK].,catfish is a [MASK].,catfish is a type of [MASK].,catfish is a kind of [MASK].
1,lemon,[produce],lemon,[produce],2,IsA,a lemon is a [MASK].,a lemon is a type of [MASK].,a lemon is a kind of [MASK].,lemon is a [MASK].,lemon is a type of [MASK].,lemon is a kind of [MASK].
2,turtle,[beast],turtles,[beasts],3,IsA,a turtle is a [MASK].,a turtle is a type of [MASK].,a turtle is a kind of [MASK].,turtles is a [MASK].,turtles is a type of [MASK].,turtles is a kind of [MASK].
3,bag,[luggage],bags,[luggages],4,IsA,a bag is a [MASK].,a bag is a type of [MASK].,a bag is a kind of [MASK].,bags is a [MASK].,bags is a type of [MASK].,bags is a kind of [MASK].
4,rabbit,[creature],rabbits,[creatures],5,IsA,a rabbit is a [MASK].,a rabbit is a type of [MASK].,a rabbit is a kind of [MASK].,rabbits is a [MASK].,rabbits is a type of [MASK].,rabbits is a kind of [MASK].


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,"[a catfish or bass is a [MASK]., a catfish or carp is a [MASK]., a catfish or shrimp is a [MASK]., a catfish or perch is a [MASK]., a catfish or alligator is a [MASK].]","[a catfish or bass is a type of [MASK]., a catfish or carp is a type of [MASK]., a catfish or shrimp is a type of [MASK]., a catfish or perch is a type of [MASK]., a catfish or alligator is a type of [MASK].]","[a catfish or bass is a kind of [MASK]., a catfish or carp is a kind of [MASK]., a catfish or shrimp is a kind of [MASK]., a catfish or perch is a kind of [MASK]., a catfish or alligator is a kind of [MASK].]","[catfish or basses is a [MASK]., catfish or carp is a [MASK]., catfish or shrimps is a [MASK]., catfish or perches is a [MASK]., catfish or alligators is a [MASK].]","[catfish or basses is a type of [MASK]., catfish or carp is a type of [MASK]., catfish or shrimps is a type of [MASK]., catfish or perches is a type of [MASK]., catfish or alligators is a type of [MASK].]","[catfish or basses is a kind of [MASK]., catfish or carp is a kind of [MASK]., catfish or shrimps is a kind of [MASK]., catfish or perches is a kind of [MASK]., catfish or alligators is a kind of [MASK].]"
1,lemon,[produce],lemon,[produce],2,IsA,"[a lemon or lime is a [MASK]., a lemon or orange is a [MASK]., a lemon or apple is a [MASK]., a lemon or yes is a [MASK]., a lemon or cherry is a [MASK].]","[a lemon or lime is a type of [MASK]., a lemon or orange is a type of [MASK]., a lemon or apple is a type of [MASK]., a lemon or yes is a type of [MASK]., a lemon or cherry is a type of [MASK].]","[a lemon or lime is a kind of [MASK]., a lemon or orange is a kind of [MASK]., a lemon or apple is a kind of [MASK]., a lemon or yes is a kind of [MASK]., a lemon or cherry is a kind of [MASK].]","[lemon or limes is a [MASK]., lemon or orange is a [MASK]., lemon or apples is a [MASK]., lemon or yes is a [MASK]., lemon or cherries is a [MASK].]","[lemon or limes is a type of [MASK]., lemon or orange is a type of [MASK]., lemon or apples is a type of [MASK]., lemon or yes is a type of [MASK]., lemon or cherries is a type of [MASK].]","[lemon or limes is a kind of [MASK]., lemon or orange is a kind of [MASK]., lemon or apples is a kind of [MASK]., lemon or yes is a kind of [MASK]., lemon or cherries is a kind of [MASK].]"
2,turtle,[beast],turtles,[beasts],3,IsA,"[a turtle or snake is a [MASK]., a turtle or bird is a [MASK]., a turtle or fish is a [MASK]., a turtle or lizard is a [MASK]., a turtle or frog is a [MASK].]","[a turtle or snake is a type of [MASK]., a turtle or bird is a type of [MASK]., a turtle or fish is a type of [MASK]., a turtle or lizard is a type of [MASK]., a turtle or frog is a type of [MASK].]","[a turtle or snake is a kind of [MASK]., a turtle or bird is a kind of [MASK]., a turtle or fish is a kind of [MASK]., a turtle or lizard is a kind of [MASK]., a turtle or frog is a kind of [MASK].]","[turtles or snakes is a [MASK]., turtles or birds is a [MASK]., turtles or fishes is a [MASK]., turtles or lizards is a [MASK]., turtles or frogs is a [MASK].]","[turtles or snakes is a type of [MASK]., turtles or birds is a type of [MASK]., turtles or fishes is a type of [MASK]., turtles or lizards is a type of [MASK]., turtles or frogs is a type of [MASK].]","[turtles or snakes is a kind of [MASK]., turtles or birds is a kind of [MASK]., turtles or fishes is a kind of [MASK]., turtles or lizards is a kind of [MASK]., turtles or frogs is a kind of [MASK].]"
3,bag,[luggage],bags,[luggages],4,IsA,"[a bag or box is a [MASK]., a bag or yes is a [MASK]., a bag or shoe is a [MASK]., a bag or case is a [MASK]., a bag or obviously is a [MASK].]","[a bag or box is a type of [MASK]., a bag or yes is a type of [MASK]., 

save ../../data/hypernymsuite/ALL/ShareSWOWStrength/consistency_rw_group/IsA.def_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/ShareSWOWStrength/consistency_rw_group/IsA.def_dap.jsonl with 2094 lines


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,a catfish is a [MASK].,a catfish is a type of [MASK].,a catfish is a kind of [MASK].,catfish is a [MASK].,catfish is a type of [MASK].,catfish is a kind of [MASK].
1,lemon,[produce],lemon,[produce],2,IsA,a lemon is a [MASK].,a lemon is a type of [MASK].,a lemon is a kind of [MASK].,lemon is a [MASK].,lemon is a type of [MASK].,lemon is a kind of [MASK].
2,turtle,[beast],turtles,[beasts],3,IsA,a turtle is a [MASK].,a turtle is a type of [MASK].,a turtle is a kind of [MASK].,turtles is a [MASK].,turtles is a type of [MASK].,turtles is a kind of [MASK].
3,bag,[luggage],bags,[luggages],4,IsA,a bag is a [MASK].,a bag is a type of [MASK].,a bag is a kind of [MASK].,bags is a [MASK].,bags is a type of [MASK].,bags is a kind of [MASK].
4,rabbit,[creature],rabbits,[creatures],5,IsA,a rabbit is a [MASK].,a rabbit is a type of [MASK].,a rabbit is a kind of [MASK].,rabbits is a [MASK].,rabbits is a type of [MASK].,rabbits is a kind of [MASK].


...

,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,a catfish is a [MASK].,a catfish is a type of [MASK].,a catfish is a kind of [MASK].,catfish is a [MASK].,catfish is a type of [MASK].,catfish is a kind of [MASK].
1,lemon,[produce],lemon,[produce],2,IsA,a lemon is a [MASK].,a lemon is a type of [MASK].,a lemon is a kind of [MASK].,lemon is a [MASK].,lemon is a type of [MASK].,lemon is a kind of [MASK].
2,turtle,[beast],turtles,[beasts],3,IsA,a turtle is a [MASK].,a turtle is a type of [MASK].,a turtle is a kind of [MASK].,turtles is a [MASK].,turtles is a type of [MASK].,turtles is a kind of [MASK].
3,bag,[luggage],bags,[luggages],4,IsA,a bag is a [MASK].,a bag is a type of [MASK].,a bag is a kind of [MASK].,bags is a [MASK].,bags is a type of [MASK].,bags is a kind of [MASK].
4,rabbit,[creature],rabbits,[creatures],5,IsA,a rabbit is a [MASK].,a rabbit is a type of [MASK].,a rabbit is a kind of [MASK].,rabbits is a [MASK].,rabbits is a type of [MASK].,rabbits is a kind of [MASK].


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,"[a catfish or bass is a [MASK]., a catfish or carp is a [MASK]., a catfish or fish is a [MASK]., a catfish or trout is a [MASK]., a catfish or shrimp is a [MASK].]","[a catfish or bass is a type of [MASK]., a catfish or carp is a type of [MASK]., a catfish or fish is a type of [MASK]., a catfish or trout is a type of [MASK]., a catfish or shrimp is a type of [MASK].]","[a catfish or bass is a kind of [MASK]., a catfish or carp is a kind of [MASK]., a catfish or fish is a kind of [MASK]., a catfish or trout is a kind of [MASK]., a catfish or shrimp is a kind of [MASK].]","[catfish or basses is a [MASK]., catfish or carp is a [MASK]., catfish or fishes is a [MASK]., catfish or trout is a [MASK]., catfish or shrimps is a [MASK].]","[catfish or basses is a type of [MASK]., catfish or carp is a type of [MASK]., catfish or fishes is a type of [MASK]., catfish or trout is a type of [MASK]., catfish or shrimps is a type of [MASK].]","[catfish or basses is a kind of [MASK]., catfish or carp is a kind of [MASK]., catfish or fishes is a kind of [MASK]., catfish or trout is a kind of [MASK]., catfish or shrimps is a kind of [MASK].]"
1,lemon,[produce],lemon,[produce],2,IsA,"[a lemon or lime is a [MASK]., a lemon or citrus is a [MASK]., a lemon or orange is a [MASK]., a lemon or grape is a [MASK]., a lemon or strawberry is a [MASK].]","[a lemon or lime is a type of [MASK]., a lemon or citrus is a type of [MASK]., a lemon or orange is a type of [MASK]., a lemon or grape is a type of [MASK]., a lemon or strawberry is a type of [MASK].]","[a lemon or lime is a kind of [MASK]., a lemon or citrus is a kind of [MASK]., a lemon or orange is a kind of [MASK]., a lemon or grape is a kind of [MASK]., a lemon or strawberry is a kind of [MASK].]","[lemon or limes is a [MASK]., lemon or citrus is a [MASK]., lemon or orange is a [MASK]., lemon or grapes is a [MASK]., lemon or strawberry is a [MASK].]","[lemon or limes is a type of [MASK]., lemon or citrus is a type of [MASK]., lemon or orange is a type of [MASK]., lemon or grapes is a type of [MASK]., lemon or strawberry is a type of [MASK].]","[lemon or limes is a kind of [MASK]., lemon or citrus is a kind of [MASK]., lemon or orange is a kind of [MASK]., lemon or grapes is a kind of [MASK]., lemon or strawberry is a kind of [MASK].]"
2,turtle,[beast],turtles,[beasts],3,IsA,"[a turtle or snake is a [MASK]., a turtle or bird is a [MASK]., a turtle or fish is a [MASK]., a turtle or lizard is a [MASK]., a turtle or frog is a [MASK].]","[a turtle or snake is a type of [MASK]., a turtle or bird is a type of [MASK]., a turtle or fish is a type of [MASK]., a turtle or lizard is a type of [MASK]., a turtle or frog is a type of [MASK].]","[a turtle or snake is a kind of [MASK]., a turtle or bird is a kind of [MASK]., a turtle or fish is a kind of [MASK]., a turtle or lizard is a kind of [MASK]., a turtle or frog is a kind of [MASK].]","[turtles or snakes is a [MASK]., turtles or birds is a [MASK]., turtles or fishes is a [MASK]., turtles or lizards is a [MASK]., turtles or frogs is a [MASK].]","[turtles or snakes is a type of [MASK]., turtles or birds is a type of [MASK]., turtles or fishes is a type of [MASK]., turtles or lizards is a type of [MASK]., turtles or frogs is a type of [MASK].]","[turtles or snakes is a kind of [MASK]., turtles or birds is a kind of [MASK]., turtles or fishes is a kind of [MASK]., turtles or lizards is a kind of [MASK]., turtles or frogs is a kind of [MASK].]"
3,bag,[luggage],bags,[luggages],4,IsA,"[a bag or luggage is a [MASK]., a bag or box is a [MASK]., a bag or suitcase is a [MASK]., a bag or backpack is a [MASK]., a bag or trunk is a [MASK].]","[a bag or luggage is a type of [MASK]., a bag or box is a ty

save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw_group/IsA.def_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw_group/IsA.def_dap.jsonl with 2094 lines


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,a catfish is a [MASK].,a catfish is a type of [MASK].,a catfish is a kind of [MASK].,catfish is a [MASK].,catfish is a type of [MASK].,catfish is a kind of [MASK].
1,lemon,[produce],lemon,[produce],2,IsA,a lemon is a [MASK].,a lemon is a type of [MASK].,a lemon is a kind of [MASK].,lemon is a [MASK].,lemon is a type of [MASK].,lemon is a kind of [MASK].
2,turtle,[beast],turtles,[beasts],3,IsA,a turtle is a [MASK].,a turtle is a type of [MASK].,a turtle is a kind of [MASK].,turtles is a [MASK].,turtles is a type of [MASK].,turtles is a kind of [MASK].
3,bag,[luggage],bags,[luggages],4,IsA,a bag is a [MASK].,a bag is a type of [MASK].,a bag is a kind of [MASK].,bags is a [MASK].,bags is a type of [MASK].,bags is a kind of [MASK].
4,rabbit,[creature],rabbits,[creatures],5,IsA,a rabbit is a [MASK].,a rabbit is a type of [MASK].,a rabbit is a kind of [MASK].,rabbits is a [MASK].,rabbits is a type of [MASK].,rabbits is a kind of [MASK].


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3
0,catfish,[fish],catfish,[fishes],1,IsA,"[a catfish or bass is a [MASK]., a catfish or carp is a [MASK]., a catfish or shrimp is a [MASK]., a catfish or perch is a [MASK]., a catfish or alligator is a [MASK].]","[a catfish or bass is a type of [MASK]., a catfish or carp is a type of [MASK]., a catfish or shrimp is a type of [MASK]., a catfish or perch is a type of [MASK]., a catfish or alligator is a type of [MASK].]","[a catfish or bass is a kind of [MASK]., a catfish or carp is a kind of [MASK]., a catfish or shrimp is a kind of [MASK]., a catfish or perch is a kind of [MASK]., a catfish or alligator is a kind of [MASK].]","[catfish or basses is a [MASK]., catfish or carp is a [MASK]., catfish or shrimps is a [MASK]., catfish or perches is a [MASK]., catfish or alligators is a [MASK].]","[catfish or basses is a type of [MASK]., catfish or carp is a type of [MASK]., catfish or shrimps is a type of [MASK]., catfish or perches is a type of [MASK]., catfish or alligators is a type of [MASK].]","[catfish or basses is a kind of [MASK]., catfish or carp is a kind of [MASK]., catfish or shrimps is a kind of [MASK]., catfish or perches is a kind of [MASK]., catfish or alligators is a kind of [MASK].]"
1,lemon,[produce],lemon,[produce],2,IsA,"[a lemon or lime is a [MASK]., a lemon or orange is a [MASK]., a lemon or yellow is a [MASK]., a lemon or apple is a [MASK]., a lemon or yes is a [MASK].]","[a lemon or lime is a type of [MASK]., a lemon or orange is a type of [MASK]., a lemon or yellow is a type of [MASK]., a lemon or apple is a type of [MASK]., a lemon or yes is a type of [MASK].]","[a lemon or lime is a kind of [MASK]., a lemon or orange is a kind of [MASK]., a lemon or yellow is a kind of [MASK]., a lemon or apple is a kind of [MASK]., a lemon or yes is a kind of [MASK].]","[lemon or limes is a [MASK]., lemon or orange is a [MASK]., lemon or yellow is a [MASK]., lemon or apples is a [MASK]., lemon or yes is a [MASK].]","[lemon or limes is a type of [MASK]., lemon or orange is a type of [MASK]., lemon or yellow is a type of [MASK]., lemon or apples is a type of [MASK]., lemon or yes is a type of [MASK].]","[lemon or limes is a kind of [MASK]., lemon or orange is a kind of [MASK]., lemon or yellow is a kind of [MASK]., lemon or apples is a kind of [MASK]., lemon or yes is a kind of [MASK].]"
2,turtle,[beast],turtles,[beasts],3,IsA,"[a turtle or snake is a [MASK]., a turtle or bird is a [MASK]., a turtle or fish is a [MASK]., a turtle or lizard is a [MASK]., a turtle or frog is a [MASK].]","[a turtle or snake is a type of [MASK]., a turtle or bird is a type of [MASK]., a turtle or fish is a type of [MASK]., a turtle or lizard is a type of [MASK]., a turtle or frog is a type of [MASK].]","[a turtle or snake is a kind of [MASK]., a turtle or bird is a kind of [MASK]., a turtle or fish is a kind of [MASK]., a turtle or lizard is a kind of [MASK]., a turtle or frog is a kind of [MASK].]","[turtles or snakes is a [MASK]., turtles or birds is a [MASK]., turtles or fishes is a [MASK]., turtles or lizards is a [MASK]., turtles or frogs is a [MASK].]","[turtles or snakes is a type of [MASK]., turtles or birds is a type of [MASK]., turtles or fishes is a type of [MASK]., turtles or lizards is a type of [MASK]., turtles or frogs is a type of [MASK].]","[turtles or snakes is a kind of [MASK]., turtles or birds is a kind of [MASK]., turtles or fishes is a kind of [MASK]., turtles or lizards is a kind of [MASK]., turtles or frogs is a kind of [MASK].]"
3,bag,[luggage],bags,[luggages],4,IsA,"[a bag or box is a [MASK]., a bag or paper is a [MASK]., a bag or yes is a [MASK]., a bag or shoe is a [MASK]., a bag or case is a [MASK].]","[a bag or box is a type of [MASK]., a bag or paper is a type of [MASK]., a bag or

## LSP (group consistency)

In [13]:
lsp_sap = [
         "[Y] such as [X].", 
         "[Y], including [X].", 
         "[Y], especially [X].", 
         "[X] or other [Y].", 
         "[X] and other [Y].", 
         "such [Y] as [X].", 
        ]

lsp_dap = [
         "[Y] such as [X] and [Z].", 
         "[Y], including [X] and [Z].", 
         "[Y], especially [X] and [Z].",  
         "[X], [Z] or other [Y].", 
         "[X], [Z] and other [Y].",
         "such [Y] as [X] and [Z].",
]

def insert_anchors(dic_sub_to_anchors, df, mask_col, sub_col, anchor_col):
    df[mask_col].head()
    df[anchor_col] = df[sub_col].apply(lambda x: dic_sub_to_anchors.get(x) )
    df[mask_col] =  df[[anchor_col, mask_col]].apply(lambda x: [ x[1].replace('[Z]', anchor)  for anchor in x[0]], axis=1)
    return df

def fill_x_into_patterns(df, patterns, ):
    '''
    sap: Y such as X 

    '''
    
    for (i, pattern) in enumerate(patterns):
        pattern_id = i+1
        df[f'mask_sentences_singular_{pattern_id}'] = df['sub_label_singular'].apply(lambda x: pattern.replace("[X]", f"{_get_article(x)} {x}").replace("[Y]", '[MASK]'))
        df[f'mask_sentences_plural_{pattern_id}'] = df['sub_label_plural'].apply(lambda x: pattern.replace("[X]", x).replace("[Y]", '[MASK]'))
        
    mask_sentences_sg_cols = [x for x in df.columns if 'mask_sentences_singular_' in x]
    mask_sentences_pl_cols = [x for x in df.columns if 'mask_sentences_plural_' in x]
    return df, mask_sentences_sg_cols, mask_sentences_pl_cols



# read the data, fill x into the placeholder and fill Y with [MASK]
data_dir = '../../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/' 
file = 'singular_plural.tsv'

for dataset, data_path in dataset_to_localpath.items():
    dic_sub_to_anchors_singular,dic_sub_to_anchors_plural = read_anchors(data_path, data_path, anchor_source='plural', debug=False)

   
    #out_dir = f'../../data/{dataset_name_to_relpath[dataset]}/consistency_group/'
    out_dir = f'../../data/hypernymsuite/ALL/{dataset}/consistency_rw_group/'
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)
    vocab_path = out_dir + 'vocab.txt'
 
    df_lsp_sap = definition_sap_singular_plural(data_path)
    
    df_lsp_sap['obj_in_BERT'] = df_lsp_sap['obj_label'].apply(lambda x: 1 if x in bert_vocab else 0)
    df_lsp_sap = df_lsp_sap.query("obj_in_BERT ==1").reset_index(drop=True)
    
    
    df_lsp_sap,mask_sentences_sg_cols, mask_sentences_pl_cols = fill_x_into_patterns(deepcopy(df_lsp_sap), patterns=lsp_sap)
    df_lsp_dap,mask_sentences_sg_cols, mask_sentences_pl_cols = fill_x_into_patterns(deepcopy(df_lsp_sap), patterns=lsp_dap)

    # fill Z with real anchors 
    for mask_col in mask_sentences_sg_cols:
        df_lsp_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_singular, df= df_lsp_dap, mask_col = mask_col, sub_col = 'sub_label_singular', anchor_col='subj_anchors_singular')

    for mask_col in mask_sentences_pl_cols:
        df_lsp_dap = insert_anchors(dic_sub_to_anchors=dic_sub_to_anchors_plural, df= df_lsp_dap, mask_col = mask_col, sub_col = 'sub_label_plural', anchor_col='subj_anchors_plural')

    # # save files 
    output_cols = ['sub_label_singular', 'obj_label_singular','sub_label_plural', 'obj_label_plural',  'uuid',
           'relation', 
           'mask_sentences_singular_1', 'mask_sentences_singular_2', 'mask_sentences_singular_3', 
           'mask_sentences_singular_4', 'mask_sentences_singular_5', 'mask_sentences_singular_6',
           'mask_sentences_plural_1', 'mask_sentences_plural_2', 'mask_sentences_plural_3', 
           'mask_sentences_plural_4', 'mask_sentences_plural_5',  'mask_sentences_plural_6']

    df_lsp_sap = df_lsp_sap[output_cols]
    df_lsp_dap = df_lsp_dap[output_cols]
    
#     out_dir = '../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/consistency_group/'
    save_dict_to_json(examples=df_lsp_sap.to_dict(orient='records'), output_path=out_dir + 'IsA.lsp_sap.jsonl') 
    save_dict_to_json(examples=df_lsp_dap.to_dict(orient='records'), output_path=out_dir + 'IsA.lsp_dap.jsonl') 
    df_lsp_sap.to_csv(out_dir + 'IsA.lsp_sap.csv')
    df_lsp_dap.to_csv(out_dir + 'IsA.lsp_dap.csv')
    display(df_lsp_sap.head())
    display(df_lsp_dap.head())

save ../../data/hypernymsuite/ALL/LM/consistency_rw_group/IsA.lsp_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/LM/consistency_rw_group/IsA.lsp_dap.jsonl with 2094 lines


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,[MASK] such as a catfish.,"[MASK], including a catfish.","[MASK], especially a catfish.",a catfish or other [MASK].,a catfish and other [MASK].,such [MASK] as a catfish.,[MASK] such as catfish.,"[MASK], including catfish.","[MASK], especially catfish.",catfish or other [MASK].,catfish and other [MASK].,such [MASK] as catfish.
1,lemon,[produce],lemon,[produce],2,IsA,[MASK] such as a lemon.,"[MASK], including a lemon.","[MASK], especially a lemon.",a lemon or other [MASK].,a lemon and other [MASK].,such [MASK] as a lemon.,[MASK] such as lemon.,"[MASK], including lemon.","[MASK], especially lemon.",lemon or other [MASK].,lemon and other [MASK].,such [MASK] as lemon.
2,turtle,[beast],turtles,[beasts],3,IsA,[MASK] such as a turtle.,"[MASK], including a turtle.","[MASK], especially a turtle.",a turtle or other [MASK].,a turtle and other [MASK].,such [MASK] as a turtle.,[MASK] such as turtles.,"[MASK], including turtles.","[MASK], especially turtles.",turtles or other [MASK].,turtles and other [MASK].,such [MASK] as turtles.
3,bag,[luggage],bags,[luggages],4,IsA,[MASK] such as a bag.,"[MASK], including a bag.","[MASK], especially a bag.",a bag or other [MASK].,a bag and other [MASK].,such [MASK] as a bag.,[MASK] such as bags.,"[MASK], including bags.","[MASK], especially bags.",bags or other [MASK].,bags and other [MASK].,such [MASK] as bags.
4,rabbit,[creature],rabbits,[creatures],5,IsA,[MASK] such as a rabbit.,"[MASK], including a rabbit.","[MASK], especially a rabbit.",a rabbit or other [MASK].,a rabbit and other [MASK].,such [MASK] as a rabbit.,[MASK] such as rabbits.,"[MASK], including rabbits.","[MASK], especially rabbits.",rabbits or other [MASK].,rabbits and other [MASK].,such [MASK] as rabbits.


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,"[[MASK] such as a catfish and bass., [MASK] such as a catfish and carp., [MASK] such as a catfish and shrimp., [MASK] such as a catfish and perch., [MASK] such as a catfish and alligator.]","[[MASK], including a catfish and bass., [MASK], including a catfish and carp., [MASK], including a catfish and shrimp., [MASK], including a catfish and perch., [MASK], including a catfish and alligator.]","[[MASK], especially a catfish and bass., [MASK], especially a catfish and carp., [MASK], especially a catfish and shrimp., [MASK], especially a catfish and perch., [MASK], especially a catfish and alligator.]","[a catfish, bass or other [MASK]., a catfish, carp or other [MASK]., a catfish, shrimp or other [MASK]., a catfish, perch or other [MASK]., a catfish, alligator or other [MASK].]","[a catfish, bass and other [MASK]., a catfish, carp and other [MASK]., a catfish, shrimp and other [MASK]., a catfish, perch and other [MASK]., a catfish, alligator and other [MASK].]","[such [MASK] as a catfish and bass., such [MASK] as a catfish and carp., such [MASK] as a catfish and shrimp., such [MASK] as a catfish and perch., such [MASK] as a catfish and alligator.]","[[MASK] such as catfish and basses., [MASK] such as catfish and carp., [MASK] such as catfish and shrimps., [MASK] such as catfish and perches., [MASK] such as catfish and alligators.]","[[MASK], including catfish and basses., [MASK], including catfish and carp., [MASK], including catfish and shrimps., [MASK], including catfish and perches., [MASK], including catfish and alligators.]","[[MASK], especially catfish and basses., [MASK], especially catfish and carp., [MASK], especially catfish and shrimps., [MASK], especially catfish and perches., [MASK], especially catfish and alligators.]","[catfish, basses or other [MASK]., catfish, carp or other [MASK]., catfish, shrimps or other [MASK]., catfish, perches or other [MASK]., catfish, alligators or other [MASK].]","[catfish, basses and other [MASK]., catfish, carp and other [MASK]., catfish, shrimps and other [MASK]., catfish, perches and other [MASK]., catfish, alligators and other [MASK].]","[such [MASK] as catfish and basses., such [MASK] as catfish and carp., such [MASK] as catfish and shrimps., such [MASK] as catfish and perches., such [MASK] as catfish and alligators.]"
1,lemon,[produce],lemon,[produce],2,IsA,"[[MASK] such as a lemon and lime., [MASK] such as a lemon and orange., [MASK] such as a lemon and apple., [MASK] such as a lemon and yes., [MASK] such as a lemon and cherry.]","[[MASK], including a lemon and lime., [MASK], including a lemon and orange., [MASK], including a lemon and apple., [MASK], including a lemon and yes., [MASK], including a lemon and cherry.]","[[MASK], especially a lemon and lime., [MASK], especially a lemon and orange., [MASK], especially a lemon and apple., [MASK], especially a lemon and yes., [MASK], especially a lemon and cherry.]","[a lemon, lime or other [MASK]., a lemon, orange or other [MASK]., a lemon, apple or other [MASK]., a lemon, yes or other [MASK]., a lemon, cherry or other [MASK].]","[a lemon, lime and other [MASK]., a lemon, orange and other [MASK]., a lemon, apple and other [MASK]., a lemon, yes and other [MASK]., a lemon, cherry and other [MASK].]","[such [MASK] as a lemon and lime., such [MASK] as a lemon and orange., such [MASK] as a lemon and apple., such [MASK] as a lemon and yes., such [MASK] as a lemon and cherry.]","[[MASK] such as lemon and limes., [MASK] such as lemon and orange., [MASK] such as lemon and apples., [MASK] such as lemon and yes., [MASK] su

save ../../data/hypernymsuite/ALL/ShareSWOWStrength/consistency_rw_group/IsA.lsp_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/ShareSWOWStrength/consistency_rw_group/IsA.lsp_dap.jsonl with 2094 lines


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,[MASK] such as a catfish.,"[MASK], including a catfish.","[MASK], especially a catfish.",a catfish or other [MASK].,a catfish and other [MASK].,such [MASK] as a catfish.,[MASK] such as catfish.,"[MASK], including catfish.","[MASK], especially catfish.",catfish or other [MASK].,catfish and other [MASK].,such [MASK] as catfish.
1,lemon,[produce],lemon,[produce],2,IsA,[MASK] such as a lemon.,"[MASK], including a lemon.","[MASK], especially a lemon.",a lemon or other [MASK].,a lemon and other [MASK].,such [MASK] as a lemon.,[MASK] such as lemon.,"[MASK], including lemon.","[MASK], especially lemon.",lemon or other [MASK].,lemon and other [MASK].,such [MASK] as lemon.
2,turtle,[beast],turtles,[beasts],3,IsA,[MASK] such as a turtle.,"[MASK], including a turtle.","[MASK], especially a turtle.",a turtle or other [MASK].,a turtle and other [MASK].,such [MASK] as a turtle.,[MASK] such as turtles.,"[MASK], including turtles.","[MASK], especially turtles.",turtles or other [MASK].,turtles and other [MASK].,such [MASK] as turtles.
3,bag,[luggage],bags,[luggages],4,IsA,[MASK] such as a bag.,"[MASK], including a bag.","[MASK], especially a bag.",a bag or other [MASK].,a bag and other [MASK].,such [MASK] as a bag.,[MASK] such as bags.,"[MASK], including bags.","[MASK], especially bags.",bags or other [MASK].,bags and other [MASK].,such [MASK] as bags.
4,rabbit,[creature],rabbits,[creatures],5,IsA,[MASK] such as a rabbit.,"[MASK], including a rabbit.","[MASK], especially a rabbit.",a rabbit or other [MASK].,a rabbit and other [MASK].,such [MASK] as a rabbit.,[MASK] such as rabbits.,"[MASK], including rabbits.","[MASK], especially rabbits.",rabbits or other [MASK].,rabbits and other [MASK].,such [MASK] as rabbits.


...

,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,[MASK] such as a catfish.,"[MASK], including a catfish.","[MASK], especially a catfish.",a catfish or other [MASK].,a catfish and other [MASK].,such [MASK] as a catfish.,[MASK] such as catfish.,"[MASK], including catfish.","[MASK], especially catfish.",catfish or other [MASK].,catfish and other [MASK].,such [MASK] as catfish.
1,lemon,[produce],lemon,[produce],2,IsA,[MASK] such as a lemon.,"[MASK], including a lemon.","[MASK], especially a lemon.",a lemon or other [MASK].,a lemon and other [MASK].,such [MASK] as a lemon.,[MASK] such as lemon.,"[MASK], including lemon.","[MASK], especially lemon.",lemon or other [MASK].,lemon and other [MASK].,such [MASK] as lemon.
2,turtle,[beast],turtles,[beasts],3,IsA,[MASK] such as a turtle.,"[MASK], including a turtle.","[MASK], especially a turtle.",a turtle or other [MASK].,a turtle and other [MASK].,such [MASK] as a turtle.,[MASK] such as turtles.,"[MASK], including turtles.","[MASK], especially turtles.",turtles or other [MASK].,turtles and other [MASK].,such [MASK] as turtles.
3,bag,[luggage],bags,[luggages],4,IsA,[MASK] such as a bag.,"[MASK], including a bag.","[MASK], especially a bag.",a bag or other [MASK].,a bag and other [MASK].,such [MASK] as a bag.,[MASK] such as bags.,"[MASK], including bags.","[MASK], especially bags.",bags or other [MASK].,bags and other [MASK].,such [MASK] as bags.
4,rabbit,[creature],rabbits,[creatures],5,IsA,[MASK] such as a rabbit.,"[MASK], including a rabbit.","[MASK], especially a rabbit.",a rabbit or other [MASK].,a rabbit and other [MASK].,such [MASK] as a rabbit.,[MASK] such as rabbits.,"[MASK], including rabbits.","[MASK], especially rabbits.",rabbits or other [MASK].,rabbits and other [MASK].,such [MASK] as rabbits.


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,"[[MASK] such as a catfish and bass., [MASK] such as a catfish and carp., [MASK] such as a catfish and fish., [MASK] such as a catfish and trout., [MASK] such as a catfish and shrimp.]","[[MASK], including a catfish and bass., [MASK], including a catfish and carp., [MASK], including a catfish and fish., [MASK], including a catfish and trout., [MASK], including a catfish and shrimp.]","[[MASK], especially a catfish and bass., [MASK], especially a catfish and carp., [MASK], especially a catfish and fish., [MASK], especially a catfish and trout., [MASK], especially a catfish and shrimp.]","[a catfish, bass or other [MASK]., a catfish, carp or other [MASK]., a catfish, fish or other [MASK]., a catfish, trout or other [MASK]., a catfish, shrimp or other [MASK].]","[a catfish, bass and other [MASK]., a catfish, carp and other [MASK]., a catfish, fish and other [MASK]., a catfish, trout and other [MASK]., a catfish, shrimp and other [MASK].]","[such [MASK] as a catfish and bass., such [MASK] as a catfish and carp., such [MASK] as a catfish and fish., such [MASK] as a catfish and trout., such [MASK] as a catfish and shrimp.]","[[MASK] such as catfish and basses., [MASK] such as catfish and carp., [MASK] such as catfish and fishes., [MASK] such as catfish and trout., [MASK] such as catfish and shrimps.]","[[MASK], including catfish and basses., [MASK], including catfish and carp., [MASK], including catfish and fishes., [MASK], including catfish and trout., [MASK], including catfish and shrimps.]","[[MASK], especially catfish and basses., [MASK], especially catfish and carp., [MASK], especially catfish and fishes., [MASK], especially catfish and trout., [MASK], especially catfish and shrimps.]","[catfish, basses or other [MASK]., catfish, carp or other [MASK]., catfish, fishes or other [MASK]., catfish, trout or other [MASK]., catfish, shrimps or other [MASK].]","[catfish, basses and other [MASK]., catfish, carp and other [MASK]., catfish, fishes and other [MASK]., catfish, trout and other [MASK]., catfish, shrimps and other [MASK].]","[such [MASK] as catfish and basses., such [MASK] as catfish and carp., such [MASK] as catfish and fishes., such [MASK] as catfish and trout., such [MASK] as catfish and shrimps.]"
1,lemon,[produce],lemon,[produce],2,IsA,"[[MASK] such as a lemon and lime., [MASK] such as a lemon and citrus., [MASK] such as a lemon and orange., [MASK] such as a lemon and grape., [MASK] such as a lemon and strawberry.]","[[MASK], including a lemon and lime., [MASK], including a lemon and citrus., [MASK], including a lemon and orange., [MASK], including a lemon and grape., [MASK], including a lemon and strawberry.]","[[MASK], especially a lemon and lime., [MASK], especially a lemon and citrus., [MASK], especially a lemon and orange., [MASK], especially a lemon and grape., [MASK], especially a lemon and strawberry.]","[a lemon, lime or other [MASK]., a lemon, citrus or other [MASK]., a lemon, orange or other [MASK]., a lemon, grape or other [MASK]., a lemon, strawberry or other [MASK].]","[a lemon, lime and other [MASK]., a lemon, citrus and other [MASK]., a lemon, orange and other [MASK]., a lemon, grape and other [MASK]., a lemon, strawberry and other [MASK].]","[such [MASK] as a lemon and lime., such [MASK] as a lemon and citrus., such [MASK] as a lemon and orange., such [MASK] as a lemon and grape., such [MASK] as a lemon and strawberry.]","[[MASK] such as lemon and limes., [MASK] such as lemon and citrus., [MASK] such as lemon and orange., [MASK] such as lemon and grapes., [MASK] such as lemon and straw

save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw_group/IsA.lsp_sap.jsonl with 2094 lines
save ../../data/hypernymsuite/ALL/AddSWOWStrength/consistency_rw_group/IsA.lsp_dap.jsonl with 2094 lines


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,[MASK] such as a catfish.,"[MASK], including a catfish.","[MASK], especially a catfish.",a catfish or other [MASK].,a catfish and other [MASK].,such [MASK] as a catfish.,[MASK] such as catfish.,"[MASK], including catfish.","[MASK], especially catfish.",catfish or other [MASK].,catfish and other [MASK].,such [MASK] as catfish.
1,lemon,[produce],lemon,[produce],2,IsA,[MASK] such as a lemon.,"[MASK], including a lemon.","[MASK], especially a lemon.",a lemon or other [MASK].,a lemon and other [MASK].,such [MASK] as a lemon.,[MASK] such as lemon.,"[MASK], including lemon.","[MASK], especially lemon.",lemon or other [MASK].,lemon and other [MASK].,such [MASK] as lemon.
2,turtle,[beast],turtles,[beasts],3,IsA,[MASK] such as a turtle.,"[MASK], including a turtle.","[MASK], especially a turtle.",a turtle or other [MASK].,a turtle and other [MASK].,such [MASK] as a turtle.,[MASK] such as turtles.,"[MASK], including turtles.","[MASK], especially turtles.",turtles or other [MASK].,turtles and other [MASK].,such [MASK] as turtles.
3,bag,[luggage],bags,[luggages],4,IsA,[MASK] such as a bag.,"[MASK], including a bag.","[MASK], especially a bag.",a bag or other [MASK].,a bag and other [MASK].,such [MASK] as a bag.,[MASK] such as bags.,"[MASK], including bags.","[MASK], especially bags.",bags or other [MASK].,bags and other [MASK].,such [MASK] as bags.
4,rabbit,[creature],rabbits,[creatures],5,IsA,[MASK] such as a rabbit.,"[MASK], including a rabbit.","[MASK], especially a rabbit.",a rabbit or other [MASK].,a rabbit and other [MASK].,such [MASK] as a rabbit.,[MASK] such as rabbits.,"[MASK], including rabbits.","[MASK], especially rabbits.",rabbits or other [MASK].,rabbits and other [MASK].,such [MASK] as rabbits.


,sub_label_singular,obj_label_singular,sub_label_plural,obj_label_plural,uuid,relation,mask_sentences_singular_1,mask_sentences_singular_2,mask_sentences_singular_3,mask_sentences_singular_4,mask_sentences_singular_5,mask_sentences_singular_6,mask_sentences_plural_1,mask_sentences_plural_2,mask_sentences_plural_3,mask_sentences_plural_4,mask_sentences_plural_5,mask_sentences_plural_6
0,catfish,[fish],catfish,[fishes],1,IsA,"[[MASK] such as a catfish and bass., [MASK] such as a catfish and carp., [MASK] such as a catfish and shrimp., [MASK] such as a catfish and perch., [MASK] such as a catfish and alligator.]","[[MASK], including a catfish and bass., [MASK], including a catfish and carp., [MASK], including a catfish and shrimp., [MASK], including a catfish and perch., [MASK], including a catfish and alligator.]","[[MASK], especially a catfish and bass., [MASK], especially a catfish and carp., [MASK], especially a catfish and shrimp., [MASK], especially a catfish and perch., [MASK], especially a catfish and alligator.]","[a catfish, bass or other [MASK]., a catfish, carp or other [MASK]., a catfish, shrimp or other [MASK]., a catfish, perch or other [MASK]., a catfish, alligator or other [MASK].]","[a catfish, bass and other [MASK]., a catfish, carp and other [MASK]., a catfish, shrimp and other [MASK]., a catfish, perch and other [MASK]., a catfish, alligator and other [MASK].]","[such [MASK] as a catfish and bass., such [MASK] as a catfish and carp., such [MASK] as a catfish and shrimp., such [MASK] as a catfish and perch., such [MASK] as a catfish and alligator.]","[[MASK] such as catfish and basses., [MASK] such as catfish and carp., [MASK] such as catfish and shrimps., [MASK] such as catfish and perches., [MASK] such as catfish and alligators.]","[[MASK], including catfish and basses., [MASK], including catfish and carp., [MASK], including catfish and shrimps., [MASK], including catfish and perches., [MASK], including catfish and alligators.]","[[MASK], especially catfish and basses., [MASK], especially catfish and carp., [MASK], especially catfish and shrimps., [MASK], especially catfish and perches., [MASK], especially catfish and alligators.]","[catfish, basses or other [MASK]., catfish, carp or other [MASK]., catfish, shrimps or other [MASK]., catfish, perches or other [MASK]., catfish, alligators or other [MASK].]","[catfish, basses and other [MASK]., catfish, carp and other [MASK]., catfish, shrimps and other [MASK]., catfish, perches and other [MASK]., catfish, alligators and other [MASK].]","[such [MASK] as catfish and basses., such [MASK] as catfish and carp., such [MASK] as catfish and shrimps., such [MASK] as catfish and perches., such [MASK] as catfish and alligators.]"
1,lemon,[produce],lemon,[produce],2,IsA,"[[MASK] such as a lemon and lime., [MASK] such as a lemon and orange., [MASK] such as a lemon and yellow., [MASK] such as a lemon and apple., [MASK] such as a lemon and yes.]","[[MASK], including a lemon and lime., [MASK], including a lemon and orange., [MASK], including a lemon and yellow., [MASK], including a lemon and apple., [MASK], including a lemon and yes.]","[[MASK], especially a lemon and lime., [MASK], especially a lemon and orange., [MASK], especially a lemon and yellow., [MASK], especially a lemon and apple., [MASK], especially a lemon and yes.]","[a lemon, lime or other [MASK]., a lemon, orange or other [MASK]., a lemon, yellow or other [MASK]., a lemon, apple or other [MASK]., a lemon, yes or other [MASK].]","[a lemon, lime and other [MASK]., a lemon, orange and other [MASK]., a lemon, yellow and other [MASK]., a lemon, apple and other [MASK]., a lemon, yes and other [MASK].]","[such [MASK] as a lemon and lime., such [MASK] as a lemon and orange., such [MASK] as a lemon and yellow., such [MASK] as a lemon and apple., such [MASK] as a lemon and yes.]","[[MASK] such as lemon and limes., [MASK] such as lemon and orange., [MASK] such as lemon and yellow., [MASK] such as lemon and apples., [MASK]

In [1]:
!scp -r ../data/hypernymsuite/SHWARTZ/consistency_group/ spartan:~/cogsci/DAP/data/hypernymsuite/SHWARTZ/

/home/chunhua/.bashrc: line 103: bind: warning: line editing not enabled
/home/chunhua/.bashrc: line 104: bind: warning: line editing not enabled
IsA.lsp_dap.jsonl                             100%   37MB   3.9MB/s   00:09    
IsA.lsp_sap.csv                               100% 4954KB   3.7MB/s   00:01    
IsA.lsp_sap.jsonl                             100%   11MB   4.0MB/s   00:02    
IsA.lsp_dap.csv                               100%   31MB   3.9MB/s   00:08    


In [ ]:
# !mkdir ../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/consistency_group/
# !scp -r ../data/probe-generalization/Syntagmatic/LM-Diagnostic-Extended/consistency_group spartan:/home/chunhua/cogsci/DAP/data/lm_diagnostic_extended/

In [7]:
# pred_col_sg= ['obj_mask_sentence_sg_1', 'obj_mask_sentence_sg_2', 'obj_mask_sentence_sg_3', 'obj_mask_sentence_sg_4', 'obj_mask_sentence_sg_5', 'obj_mask_sentence_sg_6']
# pred_col_pl =['obj_mask_sentence_pl_1', 'obj_mask_sentence_pl_2', 'obj_mask_sentence_pl_3', 'obj_mask_sentence_pl_4', 'obj_mask_sentence_pl_5', 'obj_mask_sentence_pl_6']

# for k in [1,10]:
#         # for i, (pred_col_sg, pred_col_pl) in enumerate(zip(pred_col_sg, pred_col_pl), start=1):
#     for i in range(1, 7):
#         print(f"obj_mask_sentence_sg_{i}")

obj_mask_sentence_sg_1
obj_mask_sentence_sg_2
obj_mask_sentence_sg_3
obj_mask_sentence_sg_4
obj_mask_sentence_sg_5
obj_mask_sentence_sg_6
obj_mask_sentence_sg_1
obj_mask_sentence_sg_2
obj_mask_sentence_sg_3
obj_mask_sentence_sg_4
obj_mask_sentence_sg_5
obj_mask_sentence_sg_6
